In [3]:
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from itertools import groupby, zip_longest
import pickle

In [4]:
# Si el directorio no es Seismic-Multilabel-Event-Classifier, se sale un directorio
if not os.path.basename(os.getcwd()) == 'Seismic-Multilabel-Event-Classifier':
    os.chdir('..')
    print(f"Changed directory to {os.getcwd()}")
import zipfile
zip_path = 'data/raw/Base-sismos-2024.zip'
extract_to = 'data/raw/Base-sismos-2024'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Extracted {zip_path} to {extract_to}")

Changed directory to /Users/villafuertech/Documents/Academic/University/Sexto_Semestre/Machine_Learning/Seismic-Multilabel-Event-Classifier
Extracted data/raw/Base-sismos-2024.zip to data/raw/Base-sismos-2024


In [ ]:
"""
@author: Daniel Hutabarat - UC Berkeley, 2017
"""

def processNGAfile(filepath, scalefactor=None):
    '''
    Esta función procesa un historial de aceleración de un archivo de datos NGA (.AT2)
    a un vector de una sola columna y devuelve el número total de puntos de datos y
    el intervalo de tiempo de la grabación.

    Parámetros:
    ------------
    filepath : string
        Ruta y nombre del archivo.
    scalefactor : float (opcional)
        Factor de escala que se aplica a cada componente del arreglo de aceleración.

    Salida:
    ------------
    npts: número total de puntos registrados (datos de aceleración)
    dt: intervalo de tiempo entre los puntos registrados
    time: array (n x 1) - arreglo de tiempos, misma longitud que npts
    inp_acc: array (n x 1) - arreglo de aceleraciones, misma longitud que time,
             la unidad usualmente es en g (gravedad) a menos que se indique lo contrario.

    Ejemplo (graficar tiempo vs aceleración):
    filepath = os.path.join(os.getcwd(),'motion_1')
    npts, dt, time, inp_acc = processNGAfile(filepath)
    plt.plot(time, inp_acc)
    '''
    try:
        if not scalefactor:
            scalefactor = 1.0  # Si no se proporciona, usa un factor de escala por defecto de 1.0

        with open(filepath, 'r') as f:
            content = f.readlines()  # Lee todas las líneas del archivo

        counter = 0
        desc, row4Val, acc_data = "", "", []

        for x in content:
            if counter == 3:
                # En la línea 4 se suele encontrar la información NPTS y DT
                row4Val = x
                npts_match = re.search(r'NPTS\s*=\s*([0-9.]+)', row4Val)
                if npts_match:
                    npts = float(npts_match.group(1))  # Número total de puntos
                else:
                    raise ValueError("No se encontró un valor para NPTS.")
                dt_match = re.search(r'DT\s*=\s*([0-9.]+)', row4Val)
                if dt_match:
                    dt = float(dt_match.group(1))  # Intervalo de tiempo entre puntos
                else:
                    raise ValueError("No se encontró un valor para DT.")
            elif counter == 4:
                # En la línea 5 puede comenzar la data, si no hay encabezados adicionales
                row4Val = x
                if row4Val and (row4Val[0] in ['.', '-', ' '] or row4Val[0].isdigit()):
                    print("Datos de aceleración encontrados en la línea 5.")
                    data = str(x).split()
                    for value in data:
                        a = float(value) * scalefactor
                        acc_data.append(a)
                    inp_acc = np.asarray(acc_data)
                    time = [i*dt for i in range(len(acc_data))]
            elif counter > 4:
                data = str(x).split()
                for value in data:
                    a = float(value) * scalefactor
                    acc_data.append(a)
                inp_acc = np.asarray(acc_data)
                time = [i*dt for i in range(len(acc_data))]
            counter += 1
        return npts, dt, time, inp_acc
    except IOError:
        print("¡processMotion FALLÓ!: El archivo no se encuentra en el directorio.")

In [ ]:
def procesarDatos(ruta_archivo):
    registro = []
    subcarpetas_especificas = [
        "Componente Horizontal 1",
        "Componente Horizontal 2",
        "Componente Vertical",
    ]
    for carpeta_raiz, subcarpetas, archivos in os.walk(ruta_archivo):
        registro1 = []
        for subcarpeta in subcarpetas:
            if subcarpeta in subcarpetas_especificas:
                ruta_subcarpeta = os.path.join(carpeta_raiz, subcarpeta)
                print(f"Procesando subcarpeta: {ruta_subcarpeta}")
                registros = []
                for archivo in os.listdir(ruta_subcarpeta):
                    if os.path.isfile(os.path.join(ruta_subcarpeta, archivo)):
                        ruta_archivo_full = os.path.join(ruta_subcarpeta, archivo)
                        ntps, dt, time, inp_acc = processNGAfile(ruta_archivo_full)
                        rutaS = ruta_archivo_full.split(os.sep)
                        # Ubica la última aparición de Base-sismos-2024
                        posiciones = [i for i, seg in enumerate(rutaS) if seg == 'Base-sismos-2024']
                        if not posiciones:
                            raise ValueError("No se encontró carpeta Base-sismos-2024 en la ruta")
                        base_idx = posiciones[-1]
                        falla = rutaS[base_idx + 1]
                        # Extraer magnitud
                        mag_folder = rutaS[base_idx + 2]
                        m = re.search(r'Mag\.\s*(\d+-\d+)', mag_folder)
                        if not m:
                            raise ValueError(f"No se encontró magnitud en '{mag_folder}'")
                        mag = m.group(1)
                        # Extraer Vs30
                        vs_folder = rutaS[base_idx + 3]
                        v = re.search(r'Vs30\.\s*([\d-]+)', vs_folder)
                        if not v:
                            raise ValueError(f"No se encontró Vs30 en '{vs_folder}'")
                        vs = v.group(1)
                        tipo = rutaS[base_idx + 4]
                        registros.append({
                            'Archivo': archivo,
                            'NPTS': ntps,
                            'DT': dt,
                            'Falla': falla,
                            'Mag': mag,
                            'Vs': vs,
                            'Time': time,
                            'Acc': inp_acc,
                            'Tipo': tipo
                        })
                registro1.extend(registros)
        if registro1:
            print("Registro de una carpeta completado")
            registro.append(registro1)
    return registro

In [ ]:
ruta_base = r'data/raw/Base-sismos-2024'
datos_procesados = procesarDatos(ruta_base)
print("Datos procesados y almacenados en la lista de registros.")

In [ ]:
with open('data/interim/datosML.pkl', 'wb') as f:
    pickle.dump(datos_procesados, f)

In [13]:
with open('data/interim/datosML.pkl', 'rb') as f:
    registros_por_carpeta = pickle.load(f)
print("Datos cargados desde el archivo pickle.")

Datos cargados desde el archivo pickle.


In [14]:

# Unir listas
from collections import defaultdict
resultados = []
for archivos in registros_por_carpeta:
  # Agrupar archivos según su nombre base usando defaultdict

  # defaultdict crea automáticamente listas vacías si no existe la clave
  grupos = defaultdict(list)
  for archivo in archivos:
      # Obtenemos el nombre de la primera parte del archivo
      nombre_base = archivo['Archivo'].rsplit('_', 1)[0] #Divide antes del ultimo '_'
      #print(nombre_base)
      # Añadir el archivo al grupo correspondiente
      grupos[nombre_base].append(archivo)
  # Procesar grupos
  for nombre_base, archivos_grupo in grupos.items():
      npts = archivos_grupo[0]['NPTS']
      dt = archivos_grupo[0]['DT']
      falla = archivos_grupo[0]['Falla']
      mag = archivos_grupo[0]['Mag']
      vs = archivos_grupo[0]['Vs']
      time = archivos_grupo[0]['Time']
      print(f"procesando grupo {nombre_base}")
      if len(archivos_grupo) != 3:
          print(f"Error: El grupo {nombre_base} no tiene exactamente 3 archivos.")
          for archivo in archivos_grupo:
            print(archivo['NPTS'], archivo['DT'], archivo['Falla'], archivo['Mag'], archivo['Vs'])
          print("---------")
          continue # No proceso el grupo
      # Verificar consistencia de NPTS
      if any(archivo['NPTS'] != npts for archivo in archivos_grupo):
          print(f"Error: Inconsistencia en NPTS para el grupo {nombre_base}")
          nptsTotal = []
          for archivo in archivos_grupo:
            print(archivo['NPTS'], archivo['DT'], archivo['Falla'], archivo['Mag'], archivo['Vs'], len(archivo['Time']), len(archivo['Acc']))
            nptsTotal.append(archivo['NPTS'])
          minNpts = int(min(nptsTotal))

          # Normalizo todos al mismo numero de puntos
          for archivo in archivos_grupo:
            archivo['NPTS'] = minNpts
            archivo['Time'] = archivo['Time'][:minNpts]
            archivo['Acc'] = archivo['Acc'][:minNpts]
          print("---------")
      # Comprobar consistencia del DT
      if any(archivo['DT'] != dt for archivo in archivos_grupo):
          print(f"Error: Inconsistencia en DT para el grupo {nombre_base}")
          for archivo in archivos_grupo:
            print(archivo['NPTS'], archivo['DT'], archivo['Falla'], archivo['Mag'], archivo['Vs'])
          print("---------")
          continue # No proceso el grupo
      # Comprobar tamanio grupo


      npts = archivos_grupo[0]['NPTS']
      dt = archivos_grupo[0]['DT']
      falla = archivos_grupo[0]['Falla']
      mag = archivos_grupo[0]['Mag']
      vs = archivos_grupo[0]['Vs']
      time = archivos_grupo[0]['Time']
      # Crear el diccionario agrupado
      agrupado = {
          'Archivo': nombre_base,
          'NPTS': npts,
          'DT': dt,
          'Falla': falla,
          'Mag': mag,
          'Vs': vs,
          'Time': time,
          'AccV': None,
          'AccH2': None,
          'AccH1': None
      }

      # Asignamos dependiendo del tipo de grupo
      for archivo in archivos_grupo:
          if archivo['Tipo'] == 'Componente Horizontal 1':
              agrupado['AccH1'] = archivo['Acc']
          elif archivo['Tipo'] == 'Componente Horizontal 2':
              agrupado['AccH2'] = archivo['Acc']
          elif archivo['Tipo'] == 'Componente Vertical':
              agrupado['AccV'] = archivo['Acc']
      resultados.append(agrupado)


print(f"tamanio resultados: {len(resultados)}")
  #Existen duplicados de nombre pero con distintas caracteristicas
repetidos = defaultdict(list)
# Llenamos el defaultdict con los datos
for entry in resultados:
    repetidos[entry['Archivo']].append(entry)
# Ahora, procesamos los registros duplicados
duplicados = {}
for archivo, registros in repetidos.items():
    if len(registros) > 1:
        # Si hay más de un archivo repetido, le cambiamos el nombre a uno de los registros
        for i, registro in enumerate(registros):
            if i > 0:  # Cambiar nombre al segundo archivo duplicado
                nuevo_nombre = f"{registro['Archivo']}_duplicado_{i}"
                registro['Archivo'] = nuevo_nombre
        duplicados[archivo] = registros

print(duplicados)
print(f"tamanio resultados: {len(resultados)}")



procesando grupo RSN8478_PARK2004
procesando grupo RSN8700_40204628
procesando grupo RSN8459_PARK2004
procesando grupo RSN2148_BEARCTY
procesando grupo RSN8426_BEARCTY
procesando grupo RSN8685_40204628
procesando grupo RSN424_COALINGA
procesando grupo RSN278_MAMMOTH.AH
procesando grupo RSN8317_YLINDA
procesando grupo RSN150_COYOTELK
Error: Inconsistencia en NPTS para el grupo RSN150_COYOTELK
5422.0 0.005 1 Stiker Slip (SS) 4-6 600- 5422 5422
5419.0 0.005 1 Stiker Slip (SS) 4-6 600- 5419 5419
5421.0 0.005 1 Stiker Slip (SS) 4-6 600- 5421 5421
---------
procesando grupo RSN8416_BEARCTY
procesando grupo RSN8327_YLINDA
procesando grupo RSN8451_PARK2004
procesando grupo RSN8309_CABAJA
procesando grupo RSN8484_PARK2004
procesando grupo RSN8684_40204628
procesando grupo RSN1903_BEARAFT
procesando grupo RSN8393_BEARCTY
procesando grupo RSN8680_40204628
procesando grupo RSN8328_YLINDA
procesando grupo RSN8467_PARK2004
procesando grupo RSN8377_BEARCTY
procesando grupo RSN8347_YLINDA
procesando g

In [15]:
from scipy.interpolate import interp1d

def normalizarSeries(grupos):
  dt = 0.01
  for g in grupos:
    #if g['DT'] == dt: #No es necesario pero por eficiencia
    #  continue
    print(f"Normalizando {g['Archivo']}")
    x_1 = g['AccH1']
    x_2 = g['AccH2']
    x_3 = g['AccV']
    t_orig = g['Time']
    t_new = np.arange(0, t_orig[-1] + dt, dt)
    x_1_new = interp1d(t_orig, x_1,kind = 'linear',bounds_error=False,fill_value="extrapolate")(t_new)
    x_2_new = interp1d(t_orig, x_2,kind = 'linear',bounds_error=False,fill_value="extrapolate")(t_new)
    x_3_new = interp1d(t_orig, x_3,kind = 'linear',bounds_error=False,fill_value="extrapolate")(t_new)
    g['AccH1'] = x_1_new
    g['AccH2'] = x_2_new
    g['AccV'] = x_3_new
    g['Time'] = t_new
  return grupos

In [16]:
grupos_normalizados = normalizarSeries(resultados)
print(f"tamanio grupos normalizados: {len(grupos_normalizados)}")

Normalizando RSN8478_PARK2004
Normalizando RSN8700_40204628
Normalizando RSN8459_PARK2004
Normalizando RSN2148_BEARCTY
Normalizando RSN8426_BEARCTY
Normalizando RSN8685_40204628
Normalizando RSN424_COALINGA
Normalizando RSN278_MAMMOTH.AH
Normalizando RSN8317_YLINDA
Normalizando RSN150_COYOTELK
Normalizando RSN8416_BEARCTY
Normalizando RSN8327_YLINDA
Normalizando RSN8451_PARK2004
Normalizando RSN8309_CABAJA
Normalizando RSN8484_PARK2004
Normalizando RSN8684_40204628
Normalizando RSN1903_BEARAFT
Normalizando RSN8393_BEARCTY
Normalizando RSN8680_40204628
Normalizando RSN8328_YLINDA
Normalizando RSN8467_PARK2004
Normalizando RSN8377_BEARCTY
Normalizando RSN8347_YLINDA
Normalizando RSN8332_YLINDA
Normalizando RSN2046_GILROY2
Normalizando RSN8712_40204628
Normalizando RSN8322_YLINDA
Normalizando RSN8709_40204628
Normalizando RSN8694_40204628
Normalizando RSN501_HOLLISTR
Normalizando RSN8678_40204628
Normalizando RSN8365_YLINDA
Normalizando RSN8485_PARK2004
Normalizando RSN4097_PARK2004
Norma

In [17]:
type(grupos_normalizados)

list

In [22]:
df = pd.DataFrame(grupos_normalizados)
df.head()

,Archivo,NPTS,DT,Falla,Mag,Vs,Time,AccV,AccH2,AccH1
0,RSN8478_PARK2004,32169.0,0.0100,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.3443977e-08, 3.3695554e-08, 3.3943394e-08, ...","[-5.5774385e-08, -5.6530193e-08, -5.7289814e-0...","[-1.931981e-08, -1.9326151e-08, -1.9334876e-08..."
1,RSN8700_40204628,20001.0,0.0100,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[8.5424888e-09, 8.8524636e-09, 9.1842936e-09, ...","[-2.2300064e-09, -2.3446809e-09, -2.4440089e-0...","[8.4741547e-10, 1.2447632e-09, 1.6283558e-09, ..."
2,RSN8459_PARK2004,32380.0,0.0100,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[5.855578e-08, 5.9356899e-08, 6.0156329e-08, 6...","[2.7169869e-08, 2.7004326e-08, 2.6835684e-08, ...","[1.5000493e-08, 1.4939016e-08, 1.4878223e-08, ..."
3,RSN2148_BEARCTY,8200.0,0.0050,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-2.276836e-05, -2.118971e-05, -2.810751e-05, ...","[7.43878e-06, 5.521958e-06, -2.429367e-06, -6....","[1.001939e-05, 9.63514e-06, 1.253978e-05, 1.00..."
4,RSN8426_BEARCTY,14465.0,0.0125,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.079444e-09, 3.07869528e-09, 3.07794452e-09,...","[4.3128452e-10, 4.27025392e-10, 4.22775078e-10...","[2.9098055e-09, 2.89089174e-09, 2.87200576e-09..."


In [23]:
# Eliminar DT 
df = df.drop(columns=['DT'])


In [24]:
df.head()

,Archivo,NPTS,Falla,Mag,Vs,Time,AccV,AccH2,AccH1
0,RSN8478_PARK2004,32169.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.3443977e-08, 3.3695554e-08, 3.3943394e-08, ...","[-5.5774385e-08, -5.6530193e-08, -5.7289814e-0...","[-1.931981e-08, -1.9326151e-08, -1.9334876e-08..."
1,RSN8700_40204628,20001.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[8.5424888e-09, 8.8524636e-09, 9.1842936e-09, ...","[-2.2300064e-09, -2.3446809e-09, -2.4440089e-0...","[8.4741547e-10, 1.2447632e-09, 1.6283558e-09, ..."
2,RSN8459_PARK2004,32380.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[5.855578e-08, 5.9356899e-08, 6.0156329e-08, 6...","[2.7169869e-08, 2.7004326e-08, 2.6835684e-08, ...","[1.5000493e-08, 1.4939016e-08, 1.4878223e-08, ..."
3,RSN2148_BEARCTY,8200.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[-2.276836e-05, -2.118971e-05, -2.810751e-05, ...","[7.43878e-06, 5.521958e-06, -2.429367e-06, -6....","[1.001939e-05, 9.63514e-06, 1.253978e-05, 1.00..."
4,RSN8426_BEARCTY,14465.0,1 Stiker Slip (SS),4-6,600-,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[3.079444e-09, 3.07869528e-09, 3.07794452e-09,...","[4.3128452e-10, 4.27025392e-10, 4.22775078e-10...","[2.9098055e-09, 2.89089174e-09, 2.87200576e-09..."


In [25]:
# Save the Dataframe to a json file
df.to_json('data/interim/datosML.json', orient='records', lines=True)